In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
from AutoCordon.cordon import get_road_closure_locations, split_roads_with_cordon
from pygeos.geometry import get_x, get_y
from pygeos.io import from_shapely
from pygeos.creation import linestrings
from pygeos.predicates import equals
import numpy as np

In [ ]:
centre_crossroads = linestrings([
    [(2, 2), (2, 1)],
    [(2, 2), (3, 2)],
    [(2, 2), (2, 3)],
    [(2, 2), (1, 2)]
])

ring_road = linestrings([
    [(3, 1), (3, 3)],
    [(3, 3), (1, 3)],
    [(1, 3), (1, 1)],
    [(1, 1), (3, 1)]
])

spoke_roads = linestrings([
    [(0, 4), (1, 3)],
    [(3, 3), (4, 4)],
    [(3, 1), (4, 0)],
    [(1, 1), (0, 0)]
])

dummy_roads = np.concatenate([centre_crossroads, ring_road, spoke_roads])
print(dummy_roads)

In [ ]:
dummy_roads_gdf = gpd.GeoDataFrame({"geometry": dummy_roads,
                                "id": range(len(dummy_roads))},
                                crs='epsg:27700')


In [ ]:
centre = (2, 2)
distance = 1.25
closure_locations = get_road_closure_locations(centre, distance, dummy_roads)

In [ ]:
fig, ax = plt.subplots()
dummy_roads_gdf.plot(ax=ax)
ax.scatter(get_x(closure_locations), get_y(closure_locations), color='r')
ax.scatter(*centre, color='g')
plt.show()

In [ ]:
roads = gpd.read_file(r"..\tests\data\sample_roads_soton_centre.geojson").explode().reset_index(drop=True)

In [ ]:
road_lines = from_shapely(roads.geometry)

In [ ]:
centre = (442000, 112000)
distance = 250
closure_locations = get_road_closure_locations(centre, distance, road_lines)
fig, ax = plt.subplots(figsize=[15, 15])
roads.plot(ax=ax)
ax.scatter(get_x(closure_locations), get_y(closure_locations), color='r')
ax.scatter(*centre, color='g')
plt.show()


In [ ]:
centre = (442000, 112000)
distance = 550
closure_locations = get_road_closure_locations(centre, distance, road_lines)
split_roads = split_roads_with_cordon(centre, distance, road_lines)
new_lines = [road for road in split_roads if not equals(road, road_lines).any()]

split_roads_gdf = gpd.GeoDataFrame({"geometry": new_lines, "id": range(len(new_lines))})
fig, ax = plt.subplots(figsize=[15, 15])
roads.plot(ax=ax)
split_roads_gdf.plot(color="r", ax=ax)
ax.scatter(get_x(closure_locations), get_y(closure_locations), color='y')
ax.scatter(*centre, color='g')
plt.show()